Jetsim : Road Following
===

### Step 5 from README.md

---

Right click and open 'this' notebook **JESTIM_interactive_regression.ipyn** in Google colab.

Open 'Edit/Notebook settings' toolbar in Google Colab and select **GPU**

If you get any runtime errors at anytime, go to 'Runtime' toolbar and click 'Restart Runtime'. Rerun all cells.

---

---

### Check DIY Robocar virtual track server is [ONLINE](https://www.twitch.tv/roboticists)
You should see an images of the warren track maybe with or without cars. 

---


In [ ]:
#initialize google colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
print("DONE")


Mounted at /content/gdrive
DONE


'jestim-google-colab' dirctory
===

In [ ]:
cd /content/gdrive/MyDrive/jetsim_googlecolab 

/content/gdrive/MyDrive/jetsim_googlecolab


Git clone gym-donkeycar to Google Drive
===


# Skip if already installed 

In [ ]:
##UPGRADE gym-donkeycar ONLY##
#!pip install git+https://github.com/tawnkramer/gym-donkeycar --upgrade

#!git clone https://github.com/tawnkramer/gym-donkeycar
#%cd gym-donkeycar/
#!python3 -m pip install -e .[gym-donkeycar] --user
#%cd ..
#%mv component.py gym-donkeycar/
#%mv gyminterface.py gym-donkeycar/

### IMPORTANT!!!

- Go to Google Drive and check if gym-donkeycar folder is installed. 
- gyminterface.py and component.py should be in gym-donkeycar folder.

Connect to Donkeycar simulator: (Remote only)
===

NOTE: This notebook is running locally on a Google GPU so it is not local to your network. That is why you must use 'remote' server settings to see your model perform in a simulator. 

### Method 1 : To DYI Robocar server (provided) / someone elses computer.
- Use DIY robocar server or another provided IP address and copy to remote settings in cell below.

### Method 2 : To your own host computer.
- Configure your router settings port forwarding to your host computer IP address with port 9091 to 9091.
- Google search 'my ip address' and copy to remote settings in cell below.

Create the racecar dictionary
===

---

customize your car settings.

Select track to race on. Warren is the default.

Enter the method 1 or 2 IP address as 'remote' for remote connection.

---

In [ ]:
cd ..

In [ ]:
#initialize gym wrapper 

GYM_DICT={
    
 'car':{
    #custom settings
    'racer_name': 'Jetsim_NAME',
    'bio' : 'Triton_AI_Jetsim = Jetracer',
    'country' : 'US',
    "guid": "GO_TRITON_AI",
    'body_style' : 'f1',            #car01 | f1 | donkey
    'body_rgb': (24, 43, 200),      #(R, G, B)
    'car_name' : 'Jetsim_Sriram', #displays above car
    'font_size' : 50,
    #camera settings
    "fov" : 80,
    "fish_eye_x" : 0.0, 
    "fish_eye_y" : 0.0, 
    "img_w" : 224, 
    "img_h" : 224, 
    "img_d" : 3, 
    "img_enc" : 'JPG', 
    "offset_x" : 0.0, #sides
    "offset_y" : 2.0, #height #Jetsim 2.0
    "offset_z" : 0.0, #forward
    "rot_x" : 20.0,   #tilt #Jetsim 0.0
    #"rot_y": 180,    #rotate
    },


    'default_connection': 'remote',
    'remote_connection':{
        'scene_name': 'warren',
###### REMOTE IP ADDRESS ######
        'host':  '10.2.6.144', # DIY Robocar virtual track server = 'donkey-sim.roboticist.dev'
###### REMOTE IP ADDRESS ######
        'port': 9091,
        'artificial_latency': 0}, # Besides the ping to the remote simulator, how many MORE delay would yo
    
    
    'lidar':{
        'enabled': False,
        'deg_inc': 1, # Degree increment between each ray of the lidar
        'max_range': 30.0}, # Max range of the lidar laser
    }


Send dictionary to Triton Racer Gym interface
===

### Make sure the **DIY Robocar server or Donkeycar Simulator** is ONLINE before running the code below.


In [ ]:
import time

%cd gym-donkeycar/
from gyminterface import GymInterface
gym = GymInterface(gym_config=GYM_DICT)
time.sleep(1)
imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
print(imageSeen.shape)

%cd ..

#If you run this cell more than once it will generate more cars in the sim. Turn simulator application off and back on to delete all cars. 

/content/gdrive/MyDrive/jetsim_googlecolab/gym-donkeycar


TimeoutError: ignored

---

This will initialize the **racecar in the simulator**. Check if the car **populated** in the sim.

---


In [ ]:
#If the custom car setting did not load correctly, uncomment this to take car out of sim and then re-run the cell above.

gym.onShutdown()

Import and load the model
==

In [ ]:
class Unet(nn.Module):
    def __init__(self):
        super(Unet, self).__init__()
        
        self.encode1 = nn.Sequential(nn.Conv2d(3,64,3,1,1),
                                  nn.BatchNorm2d(64),
                                  nn.ReLU(),
                                  nn.Conv2d(64,64,3,1,1),
                                  nn.BatchNorm2d(64),
                                  nn.ReLU()) 
        
        self.encode2 = nn.Sequential(nn.MaxPool2d(2,stride=2),
                                  nn.Conv2d(64,128,3,1,1),
                                  nn.BatchNorm2d(128),
                                  nn.ReLU(),
                                  nn.Conv2d(128,128,3,1,1),
                                  nn.BatchNorm2d(128),
                                  nn.ReLU())
        
        self.encode3 = nn.Sequential(nn.MaxPool2d(2,stride=2),
                                  nn.Conv2d(128,256,3,1,1),
                                  nn.BatchNorm2d(256),
                                  nn.ReLU(),
                                  nn.Conv2d(256,256,3,1,1),
                                  nn.BatchNorm2d(256),
                                  nn.ReLU())
        
        self.encode4 = nn.Sequential(nn.MaxPool2d(2,stride=2),
                                  nn.Conv2d(256,512,3,1,1),
                                  nn.BatchNorm2d(512),
                                  nn.ReLU(),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.BatchNorm2d(512),
                                  nn.ReLU())
        
        self.encode5 = nn.Sequential(nn.MaxPool2d(2,stride=2),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.BatchNorm2d(512),
                                  nn.ReLU(),
                                  nn.Conv2d(512,512,3,1,1),
                                  nn.BatchNorm2d(512),
                                  nn.ReLU())
        
        #self.upsample1 = nn.ConvTranspose2d(512,512,2,2)
        self.upsample1 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.decode1 = nn.Sequential(nn.Conv2d(1024,256,3,1,1),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(),
                                    nn.Conv2d(256,256,3,1,1),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU())
        
        #self.upsample2 = nn.ConvTranspose2d(256,256,2,2)
        self.upsample2 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.decode2 = nn.Sequential(nn.Conv2d(512,128,3,1,1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.Conv2d(128,128,3,1,1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU())
        
        #self.upsample3 = nn.ConvTranspose2d(128,128,2,2)
        self.upsample3 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.decode3 = nn.Sequential(nn.Conv2d(256,64,3,1,1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.Conv2d(64,64,3,1,1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU())
        
        #self.upsample4 = nn.ConvTranspose2d(64,64,2,2)
        self.upsample4 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.decode4 = nn.Sequential(nn.Conv2d(128,64,3,1,1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.Conv2d(64,64,3,1,1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU())
        
        self.decode5 = nn.Conv2d(64,2,1)

    def forward(self, x):
        x = torch.unbind(x,dim=1)[-1]
        x1 = self.encode1(x)
        x2 = self.encode2(x1)
        x3 = self.encode3(x2)
        x4 = self.encode4(x3)
        x5 = self.encode5(x4)
        x = self.upsample1(x5)
        x = self.decode1(torch.cat([x4,x],dim=1))
        x = self.upsample2(x)
        x = self.decode2(torch.cat([x3,x],dim=1))
        x = self.upsample3(x)
        x = self.decode3(torch.cat([x2,x],dim=1))
        x = self.upsample4(x)
        x = self.decode4(torch.cat([x1,x],dim=1))
        x = self.decode5(x)
    
        return x


In [ ]:
import torch
import torchvision

CATEGORIES = ['m1_images']
device = torch.device('cuda')
model1 = Unet().to(device)
model2 = torchvision.models.resnet18(pretrained=False)
model2.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model2 = model2.cuda().eval().half()

# Make sure the model you select is in the 'models' folder within this Google drive setup.
model1.load_state_dict(torch.load('./models/unet_weights.pth'))
model2.load_state_dict(torch.load('./models/resnet_weights.pth'))
print("DONE")

DONE


Run the model
===

In [ ]:
from utils import preprocess
import numpy as np
import time
import datetime
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output


#****************PRIMARY CONTROLS*****************
STEERING_GAIN = 2             #Scalar or intesity of steering response, usally matches THROTTLE_GAIN.
THROTTLE_GAIN = 0.65             #Scalar or intesity of throttle, usally matches STEERING_GAIN.
MAX           = 0.9             #Limiter for controlling speed.
#********************OPTIONS**********************
AUTO_RESET    = True            #reset the car to the start line if crash detected.
BOOST         = False           #Inital booster THROTTLE = 1.0 at start of race for 5 seconds.
LIMITER       = False           # THROTTLE = MAX 
PRINT         = False           #outputs steering & throttle values
STRAIGHT_AWAY = False           #Straight away for max speed based on x and z coordinate from step function.
STRAIGHT_AWAY_THROTTLE = 1.0    #Straight away max speed.
#*******************INITIAL***********************
STEERING_BIAS = 0.0             #Steering offset. 
THROTTLE_BIAS = 0.0             #Throttle offset, creeps forward at minimum speed.
BREAKS        = 0.0             #Final break value.
t0 = datetime.datetime.now()    #for lap time
out = 0                         #counter for clearing steering & throttle outputs
speed_last = 0                  #auto reset
mark = 0                        #auto reset
stuck = datetime.datetime.now() #auto reset
#*************************************************



#Main Jetsim model loop
print('JETSIM ACTIVATED')
THROTTLE = 1.0
STEERING = 0.0
try: 
    while True:
        
        #_________________________Boost_______________________
        #Inital booster THROTTLE = 1.0 at start of race for 5 seconds.
        if BOOST:
            image, x, y, z, speed, cte, lidar = gym.step(0, 1.0, 0.0, False);
            print("BOOST")
            time.sleep(5) #boost for 5 seconds
            print("DRIVING")
            BOOST = False
        #_____________________________________________________



        #___________________Image processing__________________
        #Send controls to gym wrapper and returns image, coordinates, etc.
        image, x, y, z, speed, cte, lidar = gym.step(STEERING, THROTTLE, BREAKS, False);
        image = preprocess(image).half()
        lane_out = model1(image)
        lane_pred = torch.squeeze(lane_out.max(1,keepdim=True)[1]).cpu().numpy().astype(np.uint8)
        lane_pred = Image.fromarray(lane_pred).convert('RGB')
        output = model(lane_pred).detach().cpu().numpy().flatten()
        S = float(output[0]) #-1.0 to 1.0 steering decision.
        T = float(output[1]) ##-1.0 to 1.0 throttle decision.
        #_____________________________________________________



        #_____________Steering & Throttle control_____________
        #optimize steering and throttle value. Set the limit on the top speed. 
        STEERING = round(S * STEERING_GAIN + STEERING_BIAS, 2)  
        THROTTLE = round(T * THROTTLE_GAIN + THROTTLE_BIAS, 2)
        if LIMITER: #Limiter
            if THROTTLE > MAX: #maximum speed
                THROTTLE = MAX
        #____________________________________________________



        #____________________Straight away_____________________
        #if car is on the straightest path of the track, go max speed. Warren track only because of coordinates.
        if STRAIGHT_AWAY and x <= 45 and x >= 5 and z <= -20 and z >= -30: #coordinates
            THROTTLE = STRAIGHT_AWAY_THROTTLE
            STEERING = STEERING*0.5 #reduce oversteer
        #____________________________________________________



        #____________________Print value_____________________
        #print steering and throttle value to see realtime control values.
        if PRINT: #output value
            print("Steering = "+str(STEERING)+" Throttle  = "+str(THROTTLE))
            if out == 10:
                clear_output()
                out = 0
            out+=1
        #____________________________________________________

            

        #____________________Auto reset______________________
        #reset the car to the start line if crash detected.
        if AUTO_RESET and round(speed,0) == round(speed_last,0):
          mark = datetime.datetime.now()
          check = mark - stuck
          if (str(check) > '0:00:07.000000'):
            print('NOT MOVING')
            stuck = datetime.datetime.now()
            BOOST = True
            gym.step(0.0, 0.0, 100.0, True)
            time.sleep(1)
        else: 
          speed_last = speed
          mark = stuck
          stuck = datetime.datetime.now()
        #____________________________________________________



        #___________________Show lap time___________________
        #displays by default. Cannot show steering and throttle output at the same time. 
        if x>=29 and x<=30 and z <= -20 and z >= -30:
            time_print = datetime.datetime.now() - t0
            if (str(time_print) > '0:00:01.000000'):
                print (time_print)
            t0 = datetime.datetime.now()
        #____________________________________________________


#####STOP the main loop select "Kernel" in the drop down menu and click "Interrupt Kernel"#####
except KeyboardInterrupt:
    
    #Resets steering and throttle to 0, applies breaks and resets for 1 second.
    print("!!!STOP!!!")
    gym.step(0.0, 0.0, 100.0, True);
    time.sleep(1)

JETSIM ACTIVATED
0:00:01.865441
0:00:20.498559
0:00:20.424347
0:00:20.719851
0:00:20.250297
0:00:20.448237
0:00:20.121834
0:00:22.601116
0:00:21.499440
0:00:19.836906
0:00:20.400796
0:00:19.280744
0:00:22.057043
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
0:01:27.098090
0:00:19.866074
0:00:20.331558
0:00:20.372255
0:00:17.923825
0:00:24.723419
0:00:20.053474
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
0:03:17.639861
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting car...
BOOST
DRIVING
0:00:59.573054
0:00:21.895539
0:00:20.503168
0:00:20.916790
NOT MOVING
Resetting car...
BOOST
DRIVING
NOT MOVING
Resetting

In [ ]:
#RESET run this cell multiple times to stop the car if it still running.
gym.step(0.0, 0.0, 100.0, True);

If finished - YOU MUST Take car out of SERVER!!!
===

In [ ]:
gym.onShutdown()